# 非凸优化：SGD的opt method

关注：当使用了SGD后，BGD中使用的优化工具效果变化，以及需要哪些新的工具来处理SGD中增加的随机性

### I. SGD相对BGD的优点和问题

- **SGD最重要的优点：运算效率**
    - sample中有大量相似样本时，具有明显的效率优势
        - 极端假如1：sample中所有的样本是同一个样本的重复，那么GD一次迭代用掉的时间是SGD的N倍，但是效果一样
        - 放松假设2：所有sample set中样本都是取值很接近的样本，SGD的优势也同样明显
        - 正常假设3：一般的sample set中都有不少样本互相很接近，即使总体上很分散，SGD的效率优势也不会完全消失
    - 参考前面凸条件下的收敛分析，虽然看起来SGD是$O(\frac{1}{ε})$，不如Batch GD的$O(log(\frac{1}{ε}))$，但BGD迭代1次的计算量大约是SGD迭代N次的计算量，N是样本数量

- **问题1：抽样过程本身如果不能足够random的话，收敛到poor local min: shuffle**
    - **solution**: shuffle every epoch

- **问题2：SGD的variance很大，导致收敛路径fluctuation: lr decay, mini-batch**
    - 分析：
        - $Var_{SGD}=Var(div(f_{i}, d_{i}))=Var(div(f, d))$
        - $Var_{BGD}=Var(\frac{1}{N}\underset{i}{Σ}div(f_{i}, d_{i}))=\frac{1}{N}Var(div(f, d))$
        - 所以，单个样本的variance of divergence是样本整体的N倍
        - 优化时，SGD以$min\ div(fi, di)$为目标，BGD以$min\ \frac{1}{N}\underset{i}{Σ}div(fi, di)$为目标，必然导致SGD的优化目标本身不稳定，从而优化结果也不稳定，而BGD的优化目标要稳定得多。
            - 这表现在结果上可以看test error curve。用validation set画出来的test error curve图中，SGD的test error下降路径的波动要大很多
        - 注意：这里比较的variance不是用validation set画出来的test error的波动性。这里的variance是test error波动性产生的原因。这个variance衡量了训练时抽样导致的目标函数波动性。
    - **solution**：
        - **mini-batch GD**: 由于倒数函数特性，在N较小的时候$V_{SGD}与V_{BGD}$的差异缩小的较快，随着N的增大，不同batch大小的variance差异会减少。因此，mini-batch是很好的解决方式。
        - **lr decay**: **如果不用lr decay的话无法收敛，用了lr decay的收敛结果也不好**
    - 区分SGD中的variance和batchnorm处理的variance：
        - **与batchnorm的关系**：batchnorm主要配合mini-batch使用。mini-batch SGD带来的variance和batchnorm要解决的covariate shift中的variance是不同来源。后者关注每个layer的input分布；前者可以视为处理的时在相同分布的input中抽样导致的随机性。batchnorm在mini-batch SGD和BGD中发挥同样的作用。但是不能用于单样本的SGD。

### II. stochastic optimization method
- *momentum和adagrad，以及后来的RMSprop，Adam等都是为SGD提出的优化方法。*他们的一个核心特点就是用了**exponentially weighted moving average:**
    - 以Momentum为例，SGD的running average of single sample gradient是overall gradient of the entire training set的近似。
    - 在RMSprop中，running average of squared gradient则是对2nd raw moment(the uncentered variance) of the gradient的估计。

#### II. 1 GD优化过程分析
layer：$W_{l}^{[t+1]}=W_{l}^{[t]}-η\bigtriangledown_{W_{l}}Loss$ ,  $W_{l}$指第$l$层的weight matrix \
neuron：$w_{l, i}^{[t+1]}=w_{l, i}^{[t]}-η\bigtriangledown_{w_{l, l}}Loss$ , $w_{l,i}$指$l$层$ith$ neuron的weight vector \

**2个关键控制变量**：
1. **learning rate：**控制step size：η没有方向，只有大小
2. **梯度：**同时控制方向和大小，但它提供的方向信息更重要。理解$\bigtriangledown_{w_{l, i}}Loss$方向信息：\
(1) 假设$w_{l, i}$的维度是d，那么$\bigtriangledown_{w_{l, i}}Loss$的维度也是d \
(2) d个元素中每个都有+/-号信息，表示梯度在对应维度上的方向，所以单个维度的方向只有正负两种 \
(3) d个元素的方向和大小一起决定梯度在整个空间中的方向。以d=2为例，$\bigtriangledown_{w_{l, i}}Loss$在平面上的方向由它在两个维度上各自的方向和大小决定：\
[这时候，两个维度汇总后的梯度方向也是等高线的法线方向]
<img src="pics/gdopt.png" alt="alt text" width="300"/> \
(4) 注意区分函数在x点处的**梯度方向**和函数在x点处的**切线/面方向** \
a. 一维函数：梯度方向是$f^{'}(x)$的符号+或-，切线方向是$tagθ=f^{'}(x)$的θ \
b. 二维函数：梯度方向是等高线法线方向，切面方向

#### II. 2 Momentum
1. 要解决的问题：减少收敛路径在某些维度上的**oscillate**，增加learning rate，加速收敛
2. 分析思路：\
(1) oscillate产生的原因，单个维度上的梯度方向+/-交叠 \
(2) 可以减轻这一现象的方式：取历史平均可以消减这种+/-交叠
3. 解决办法：用gradient的历史加权平均来代替梯度，主要是控制方向，也含有step size信息

4. 算法 \
第一种迭代式
$$
\begin{align}
v_0 & = 0 \\
v_{t+1} & = \rho v_t+\nabla f(w_t) \ \#当前步是梯度的历史加权\\
w_{t+1} & = w_t-\alpha v_{t+1}
\end{align}
$$
第二种等价迭代式：
$$
\begin{align}
v_{t+1} & = \beta v_t+(1-\beta )\nabla f(w_t) ...[1]  \\
w_{t+1} & = w_t-\gamma v_{t+1} ...[2]\\
\end{align}
$$
第三种等价迭代式：
$$
\begin{align}
v_{t+1} & = \rho v_t-\alpha \nabla f(w_t)   \\
w_{t+1} & = w_t+v_{t+1} \\
\end{align}
$$

4. (续)算法等价性说明 \
<font color=gray>(1) 第一种和第二种迭代式等价：\
取$\rho=\frac{\beta}{1-\beta}$,有$\frac{1}{1-\beta}=1+\rho$。代入[1]式有:
$$
\begin{align}
\frac{1}{(1-\beta )} v_{t+1} & = \frac{\beta }{(1-\beta )}v_t+\nabla f(w_t) \\
(1+\rho) v_{t+1} & = \rho v_t + \nabla f(w_t)...[3] \\
\end{align}
$$
$取\gamma=\alpha(1+\rho)$，代入[2]式有：
$$
w_{t+1} = w_t-\alpha(1+\rho)v_{t+1}...[4]
$$
可见，[3]和[4]与第一种迭代式相同。</font> \
<font color=gray>(2) 第一种和第三种迭代式等价：直接将v代入w的迭代式展开即可。</font>

5. **理解历史加权：exponentially weighted moving average** \
(1) 递推式 \
$$
\begin{align}
v_{0} & =0 \\
v_{1} & =β*v_{0}+(1-β)*θ_{1} \\
v_{2} & =β*v_{1}+(1-β)*θ_{2}=β^{2}*v_{0}+β*(1-β)*θ_{1}+(1-β)*θ_{2} \\
v_{3} & =β*v_{2}+(1-β)*θ_{3} \\
& = β^{3}*v_{0}+β^{2}*(1-β)*θ_{1}+β*(1-β)*θ_{2}+(1-β)*θ_{3}\\
 …     \\ 
v_{t} & =β*v_{t-1}+(1-β)*θ_{t} \\
& =(1-β)*(β^{t-1}*θ_{1}+β^{t-2}*θ_{2}+...+β^{1}*θ_{t-1}+β^{0}*θ_{t}) \\
& =(1-β)*[(β^{t-1}, β^{t-2}, ..., β^{0})\otimes(θ_{1}, θ_{2}, ..., θ_{t})]
\end{align}
$$
(2) 递推式特点：
- t时点，距离k个时段的$θ_{t-k}$对$v_{t}$的贡献大小为$(1-β)*β^{k}$
- 一种粗略的计算方式是，$v_{t}$主要用的是过去$\frac{1}{1-β}$个时间段的数据 \
(3) 偏差修正
- 问题：由于$v_{0}=0$，导致初始状态的加权平均结果严重偏小
- 修正方式：
$$
\begin{align}
v^{'}_{t}&  =\frac{v_{t}}{1-β^{t}} \\
1-β^{t}& =(1-β)*(β^{t-1}, β^{t-2}, ..., β^{0}) \\
\frac{v_{t}}{1-β^{t}}& =\frac{(β^{t-1}*θ_{1}+β^{t-2}*θ_{2}+...+β^{1}*θ_{t-1}+β^{0}*θ_{t})}{(β^{t-1}+ β^{t-2}+ ...+ β^{0})}
\end{align}
$$
此时LHS是$θ_{1}, ..., θ_{t}$的加权平均
 - β默认取0.9，*原因见”why moments really works”。*在凸条件下，condition number越大，β的最优取值越大。

6. **尚未解决的优化问题**：原本单一learning rate只能迁就Hessian对应的$\frac{1}{maxλ_{i}}$，导致迭代速度慢。用了momentum之后，虽然可以放大learning rate，但是每个维度仍然使用相同的learning rate。

#### II. 3 AdaGrad
1. **要解决的问题**：加速SGD中**sparse feature**的收敛 \
<font color=blue>论文中提出的具体问题是结合应用场景的：加速sparse feature对应的weight的迭代速度。</font>\
(1) feature有dense feature和sparse feature两种，前一种经常出现，后一种则出现频次低。一个sample对应的input中，没有出现的feature取值为0。以nlp的bag of words模型为例，如果每个句子是一个input vector，上面很多element取值对应的是罕见词，大多数时候都是0，而与常用词对应的feature则经常取1。\
(2) 用SGD训练时，取0的feature无法让对应的weight进行迭代，只有取1的feature才能让对应的weight正常迭代。这就带来问题：常见词对应的weight迭代较快，sparse feature对应的weight则迭代很慢。即使用了momentum，sparse features也会因为平均gradient很小而迭代的很慢。

2. **改进思路**：用adaptive learning rate加速收敛速度。放大sparse feature的learning rate，缩小dense feature的learning rate。背后的intuition：每次出现sparse feature，就要take notice。

3. **实现方式**：每个维度的learning rate与该维度上的sum of squared gradient成反比。\
(1) 预期的效果，当某个维度上的$|g_{i,t}|$均值较大值时(A图)，降低step size；反之，当$|g_{i,t}|$的均值较小时(B图)，提高step size。
<img src="pics/adagrad.png" alt="alt text" width="360"/>\
(2) <font color=green>**[rk’s note]为什么用累积历史值而非当前值**：如果用当前值，那些sparse feature在好不容易拿到非零input的时候，不会受惠会当前梯度大小，因为当前它的梯度本身可能较大。而这个算法的目的就是，不论当前梯度大还是小，都要补偿sparse feature维度上的step size。而能将sparse feature和dense feature拉开差距的数据就是他们的梯度的历史值。</font>\
(3) <font color=norange>**[rk's note]为什么不用sum of absolute gradient，而要用root of sum of squared gradient：**后者更smooth，有更好的优化性质</font>

#### II. 4 RMSprop
**要解决的问题**：AdaGrad中adaptive learning rate随时间累积，当迭代次数过高时，会导致step size→0

**思路**：用running average of squared gradient来代替Adagrad中的sum of squared gradient. 
 - 在SGD条件下就用running average of squared gradient来近似uncentered variance。
1. hinton的观点：RMSprop是在Rprop的基础上改造以适用于SGD，但是**我认为这里不合理**\
(1) 因为RProp的核心是用梯度的方向信息来调节learning rate大小，完全不用gradient信息。但是RMSProp则完全不用方向信息，只是用了gradien大小的信息。所以我更倾向于RMSProp是对AdaGrad的修改。\
(2) RProp：$w^{[t]}_{l,i}=w{[t-1]}-η*g_{l, i}*\nabla_{w_{l,i}}L(w)$
 如果连续两次gradient的符号是同号++或- -，那么将$g_{l, i}=g_{l, i}*1.2$
                - 如果连续两次gradient的符号是反号+-或-+，那么$g_{l, i}=g_{l, i}*0.5$
<img src="pics/hinton1.png" alt="alt text" width="560"/>
 - Hinton的说明：RProp→RmsProp
<img src="pics/hinton2.png" alt="alt text" width="560"/>

2. adam论文的观点：用running average of squared gradient来近似uncentered variance(2nd moment) of the gradient。\
*[rk’s补充]相当于假设了BGD条件下用uncentered variance of gradient的倒数来调节stepsize的合理性，然后考虑到SGD场景，就用adjusted exponential average  of squared gradient。*
$$
\begin{align}
\frac{1}{N}Σx_{i}^2 & ＝uncentered\ \ variance\\
\frac{1}{N}Σ(x_{i}-\bar x)^2 & ＝variance
\end{align}
$$
**[rk’s note]为什么要estimate uncentered variance而不estimate centered variance of gradient，后者看起来更接近Hessian的定义**\
a. variance of gradient不是Hessian，Hessian的定义是gradient of gradient所以后者并不会更接近Hessian \
b. 如果梯度的历史均值$\bar{g}$在0附近，那么两者大小接近(下图中C)；如果$\bar{g}$>0(下图中A)或$\bar{g}$<0(下图中B)，那么$(g-\bar{g})^{2}<g^2$ \
c. 从收敛目标来看，如果g的均值偏正较多或者偏负较多，大多数时候$|g|>0$，此时，为了减少divergence，会更希望learning rate较小，而$\frac{1}{g^2}<\frac{1}{(g-\bar{g})^2}$，因此用uncentered variance是更好的选择。
<img src="pics/adam.png" alt="alt text" width="300"/>

3. 11-785课上的观点：running average of squared gradient是对二阶梯度的近似估计，但是**我认为这里不合理，或者说这只是一种非常模糊意义上的近似** \
*[rk’s note]*$\frac{\partial L^{2}}{\partial^{2} w}\approx (\frac{\partial L}{\partial w})^{2}$很难成立.\
(1) 参考凸条件下，η取Hessian特征值的倒数，一个neuron对应的vector $w_{l,i}$的hessian matrix是$H_{l,i}=\frac{\partial L^{2}}{\partial^{2} w_{l,i}}$，将$w_{l,i}$简单记为$w$，其对应的hessian记为$H=\frac{\partial L^{2}}{\partial^{2} w}$，则w迭代时各维度上的learning rate最优值是：$η_{k}=\frac{1}{λ_{k}}$ \
(2) 第一个简化：假设w各个维度之间不相关，也就是只关注H对角线上的元素。取$η_{k}=\frac{1}{h_{kk}}$。此时可以不计算reverse Hessian，但还是要计算Hessian \
(3) 第二个简化：$diag(H(w))=\frac{\partial L^{2}}{\partial^{2} w}$的估计可以用$\frac{\partial L^{2}}{\partial^{2} w}\approx (\frac{\partial L}{\partial w})^{2}$
- ***说明：在多元正态分布中，reverse Hessian就是covariate matrix of w。H对角线上的元素是variance of w，也是2nd derivatie of w，但不是variance of gradient  of w。***

**实现方式**：用adjusted exponential average of squared gradient来调节learning rate \
 · exponential average of squared gradient of the mini-batch(or single sample)可以视为estimate of squared gradient of the overall set。

<font color=red>**注：MSprop不能替代momentum。**</font> \
因为RMSprop只调整learning rate，不会影响前进的方向，而momentum会影响。反之，在momentum中各维度learning rate相同，加速收敛依靠一定程度上增加所有维度的learning rate。考虑到cross ravine方向上不能有太大的step size，因此还是会对learning rate有限制。RMSprop进一步减少了cross ravine方向的step size，放大了gradient较小的方向上的step size。

**[rk’s note]从数学上理解：**
1. $lr=\frac{η}{\sqrt{M_{2}(w)+ε}}$，其中$M_{2}^{[t+1]}(w)=β*M_{2}^{[t]}(w)+(\nabla_{w} L)^{2}$ \
2. 可见，决定lr大小的是$|\nabla_{w} L|$的历史值，而不受其方向影响。其作用方式是:\
(1) 如果$|\nabla_{w} L|$的值一直很大，那么step size减少 \
(2) 如果一直很小，step size放大 \
(3) 如果在过去几步中经历了由大到小，那么step size会比之前放大一些 \
(4) 果在过去几步中经历了由小到大，那么step size会比之前缩小一些 \
3. 为什么要用历史加权：因为是用在SGD中，用历史加权平均相当于在降低对$|\nabla_{w} L|$估计的方差，所以才能更好的估计整个样本的uncentered variance。